# Data Ingest

Access data stored on s3 and explore with Athena

In [51]:
import warnings
warnings.simplefilter(action="ignore", category=UserWarning)

import boto3
import sagemaker
from pyathena import connect
import pandas as pd

Set up sagemaker instance

In [53]:
sess = sagemaker.Session()
bucket = 'ads-508-final'
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

Verify objects in public bucket

In [48]:
!aws s3 ls s3://ads-508-final/playstation/

                           PRE games/
2025-03-14 05:41:16   70127573 achievements.csv
2025-03-14 05:41:16  799125767 history.csv
2025-03-14 05:41:31   10705226 players.csv
2025-03-14 05:41:18    2320804 prices.csv
2025-03-14 05:41:16   74376997 purchased_games.csv


In [29]:
# set path for public bucket
s3_playstation_path_public = 's3://ads-508-final/playstation/'

In [ ]:
# set path for private bucket


In [8]:
# set staging directory
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [9]:
# create connection with pyathena
# EXPLORE USING ENGINE
conn = connect(region_name = region, s3_staging_dir = s3_staging_dir)

In [7]:
# define database name
database_name = "playstationaws"

In [10]:
# create database
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement) 

CREATE DATABASE IF NOT EXISTS playstationaws


In [12]:
# execute query to create database
pd.read_sql(statement, conn)

/tmp/ipykernel_1242/3803073958.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [13]:
# execute query to show database is present
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head()

/tmp/ipykernel_1242/2646085860.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,database_name
0,default
1,playstationaws


In [14]:
# define table names based on datasets
table_names = ['achievements', 'games', 'history', 'players', 'prices', 'purchased_games']

In [44]:
# create table for games data
statement = """create external table if not exists {}.{} (
            game_id string,
            title string,
            platform string,
            developers string,
            publishers string,
            genres string,
            supported_languages string,
            release_date date
        )
        row format delimited
        fields terminated by ','
        lines terminated by '\\n'
        location '{}games/'
        tblproperties ('compressionType'='gzip', 'skip.header.line.count'='1');""".format(
            database_name, table_names[1], s3_playstation_path
        )
print(statement)

create external table if not exists playstationaws.games (
            game_id string,
            title string,
            platform string,
            developers string,
            publishers string,
            genres string,
            supported_languages string,
            release_date date
        )
        row format delimited
        fields terminated by ','
        lines terminated by '\n'
        location 's3://ads-508-final/playstation/games'
        tblproperties ('compressionType'='gzip', 'skip.header.line.count'='1');


In [45]:
# execute query to create table and load data
pd.read_sql(statement, conn)

/tmp/ipykernel_1242/618921652.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [46]:
# define query and check load
statement = """select * from {}.games limit 10""".format(database_name)

In [47]:
df = pd.read_sql(statement, conn)
df.head()

/tmp/ipykernel_1242/1351440842.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,game_id,title,platform,developers,publishers,genres,supported_languages,release_date
0,749375,Medieval Match Master,PS4,['Erik Games'],['Erik Games'],['Puzzle'],,2024-11-22
1,749839,Sunforge Solar Survival,PS4,['GrizzlyGames'],['GrizzlyGames'],,,2024-11-21
2,749347,Find Love Or Die Trying,PS5,['Sunseeker Games'],['Ratalaika Games'],['Visual Novel'],,2024-11-22
3,749341,Medieval Match Master,PS4,['Erik Games'],['Erik Games'],['Puzzle'],,2024-11-22
4,749097,Bust A Block 2,PS5,['ThiGames'],['ThiGames'],['Puzzle'],,2024-11-21
